In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem.snowball import SnowballStemmer
# Use English stemmer.
stemmer = SnowballStemmer("english")
from sklearn.metrics import log_loss
# log_loss(y_test, preds)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from gensim.models import KeyedVectors
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('train.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train['is_duplicate'].value_counts()

In [ ]:
sns.countplot(x='is_duplicate', data=df_train)

In [ ]:
final_list = list(set(df_train['qid1']) | set(df_train['qid1']))

In [ ]:
print('Total number of unique questions : ', len(final_list))

In [ ]:
qids = pd.Series(df_train['qid1'].tolist() + df_train['qid2'].tolist())

plt.figure(figsize=(15, 5))
plt.hist(qids.value_counts(), bins=50)
plt.yscale('log', nonposy='clip')
plt.title('Log-Histogram of question appearance counts')
plt.xlabel('Number of occurences of question')
plt.ylabel('Number of questions')
print()

In [ ]:
 df_train['question1'].isna().sum(), df_train['question2'].isna().sum()

In [ ]:
df_train.shape[0] - df_train.dropna(how='any').shape[0]

In [ ]:
df_train = df_train.dropna(how='any')

In [ ]:
df_train['question1'] = df_train['question1'].str.lower()
df_train['question2'] = df_train['question2'].str.lower()

In [ ]:
df_train[:5]['question1']

In [ ]:
import string
exclude = set(string.punctuation)

In [ ]:
df_train['qmark_count1'] = df_train['question1'].apply(lambda x: len([item for item in x if item == '?']))

In [ ]:
df_train['qmark_count2'] = df_train['question2'].apply(lambda x: len([item for item in x if item == '?']))

In [ ]:
df_train['question1'] = df_train['question1'].apply(lambda x: ''.join(item for item in x if item not in exclude))
df_train['question2'] = df_train['question2'].apply(lambda x: ''.join(item for item in x if item not in exclude))

In [ ]:
df_train[:5]['question1']

In [ ]:
df_train['question1'] = df_train['question1'].str.split()
df_train['question2'] = df_train['question2'].str.split()

In [ ]:
x = df_train[(df_train['question1'].str.len() == 0) | (df_train['question2'].str.len() == 0)]
x.shape

In [ ]:
df_train = df_train[(df_train['question1'].str.len() != 0) & (df_train['question2'].str.len() != 0)]

In [ ]:
common_words = []
for a,b in zip(df_train.question1, df_train.question2): 
    t = len(list(set(a) & set(b)))
    common_words.append(t)

In [ ]:
df_train['common_words'] = common_words

In [ ]:
df_train['question1'] = df_train['question1'].apply(lambda x: [item for item in x if item not in stop])
df_train['question2'] = df_train['question2'].apply(lambda x: [item for item in x if item not in stop])

In [ ]:
df_train[:5]['question1']

In [ ]:
df_train['q1_s'] = df_train['question1'].apply(lambda x: [stemmer.stem(y) for y in x])
df_train['q2_s'] = df_train['question2'].apply(lambda x: [stemmer.stem(y) for y in x])

In [ ]:
df_train[:5]['q1_s']

In [ ]:
df_train[:5]['q2_s']

In [ ]:
df_train[:5]['question1']

In [ ]:
df_train.to_pickle("dataset.pkl")

In [ ]:
x = df_train[(df_train['q1_s'].str.len() == 0) | (df_train['q2_s'].str.len() == 0)]
x.shape

In [ ]:
x['is_duplicate'].value_counts()

In [ ]:
df_train = df_train[(df_train['q1_s'].str.len() != 0) & (df_train['q2_s'].str.len() != 0)]

In [ ]:
match_frac = []
for a,b in zip(df_train.q1_s, df_train.q2_s): 
    t = len(list(set(a) & set(b)))/(len(set(a)) + len(set(b)))
    match_frac.append(t)

In [ ]:
df_train['match_frac'] = match_frac

In [ ]:
lengths = []
for a,b in zip(df_train.q1_s, df_train.q2_s): 
    t = abs(len(a) - len(b))
    lengths.append(t)

In [ ]:
df_train['len_diff'] = lengths

In [ ]:
plt.figure(figsize=(15, 3))
plt.hist(df_train[df_train['is_duplicate'] == 0]['match_frac'], bins=20, normed=True, label='Not Duplicate')
plt.hist(df_train[df_train['is_duplicate'] == 1]['match_frac'], bins=20, normed=True, alpha=0.7, label='Duplicate')
plt.legend()
plt.title('Label distribution over word_match_share', fontsize=15)
plt.xlabel('word_match_share', fontsize=15)

In [ ]:
# df_train.to_pickle("dataset.pkl")

In [ ]:
sentences = list(df_train['q1_s']) + list(df_train['q2_s'])

embed_size = 200

model = Word2Vec(sentences, size=embed_size, window=3, min_count=1, workers=4)

In [ ]:
model.wv.most_similar(positive=['water'])

In [ ]:
with open('word2vec.pkl', 'wb') as fid:
    pickle.dump(model, fid)    

In [ ]:
df_train.to_pickle('dataset.pkl')

In [ ]:
%reset

In [ ]:
import pickle

In [ ]:
# load it again
with open('word2vec.pkl', 'rb') as fid:
    word2vec = pickle.load(fid)

In [ ]:
df_train = pd.read_pickle('dataset.pkl')

In [ ]:
df_train.drop(['id', 'qid1', 'qid2', 'question1', 'question2', 'q1_s', 'q2_s'], axis=1, inplace=True)

In [ ]:
Y = df_train['is_duplicate']

In [ ]:
df_train.drop(['is_duplicate'], inplace=True, axis=1)

In [ ]:
# X = df_train[['common_words', 'match_frac', 'len_diff', 'cosine_sim_q1_q2']]

In [ ]:
df_train = normalize(df_train)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train, Y, test_size=0.20, random_state=42)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)

X_train, y_train = ros.fit_resample(X_train, y_train)

In [ ]:
y_train.value_counts()

In [ ]:
def conf_matrix(true, preds):
    cm = confusion_matrix(y_test, preds, normalize='true')
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
    ax.set_title('Confusion Matrix'); 

In [ ]:
clf = LogisticRegression(class_weight={0: 0.4, 1: 0.6}, max_iter=300)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_test)

In [ ]:
print("Accuracy : ", accuracy_score(y_test, preds))
print("Precision : ", precision_score(y_test, preds))
print("Recall : ", recall_score(y_test, preds))

In [ ]:
conf_matrix(y_test, preds)

In [ ]:
prec, rec = [], []
for weight in np.arange(0.05, 1.0, 0.05):
    clf = LogisticRegression(class_weight={0: 1-weight, 1: weight}, max_iter=300)
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    prec.append(precision_score(y_test, preds))
    rec.append(recall_score(y_test, preds))
#     print("Accuracy : ", accuracy_score(y_test, preds))
#     print("Precision : ", precision_score(y_test, preds))
#     print("Recall : ", recall_score(y_test, preds))

In [ ]:
plt.figure(figsize=(15, 4))
plt.title('Precision vs Recall for different class weights')
plt.xlabel('Number of occurences of question')
plt.ylabel('Number of questions')
ax = sns.lineplot(x=prec[6:], y=rec[6:])
# ax.text("An annotation", horizontalalignment='left', size='medium', color='black', weight='semibold')

so with simple features like common words, length diff etc, we can see that our precision and recall are not that good. We probably need more information to be able to classify correctly. We can also notice the variation in precision and recall, as we change the class weights. Since we have imbalanced data, if we increase the weight of minority class, it outperforms majority class, and in the process we end up marking non duplicate questions as well duplicate (low precision). Low precision is absolutely undesirable

## Adding word vectors features

In [ ]:
global vecs
vecs = []
global vecs2
vecs2 = []
embed_size = 200
def cosine_sim_ques(q1, q2):
    global vecs
    global vecs2
    
    for i in list(q1.values):
        vec = np.zeros(embed_size,)
        for x in i:
            vec = vec + model[x]
        vec = vec/len(i)
        vecs.append(vec)

    vecs = np.array(vecs)
    vecs[np.isnan(vecs)] = 0

    for i in list(q2):
        vec = np.zeros(embed_size,)
        for x in i:
            vec = vec + model[x]
        vec = vec/len(i)
        vecs2.append(vec)

    vecs2 = np.array(vecs2)
    vecs2[np.isnan(vecs2)] = 0    
    c = []
    for i in range(vecs.shape[0]):
        result = 1 - spatial.distance.cosine(vecs[i], vecs2[i])
        c.append(result)
    print(len(c))
    return c

In [ ]:
df_train['cosine_sim_q1_q2'] = cosine_sim_ques(df_train['q1_s'], df_train['q2_s'])

In [ ]:
vecs.shape, vecs2.shape

In [ ]:
vecs = pd.DataFrame(vecs)
vecs2 = pd.DataFrame(vecs2)

In [ ]:
df_train.shape

In [ ]:
vecs.reset_index(drop=True, inplace=True)

vecs2.reset_index(drop=True, inplace=True)

df_train.reset_index(drop=True, inplace=True)

df_train = pd.concat((df_train, vecs), axis=1)

df_train.reset_index(drop=True, inplace=True)

df_train = pd.concat((df_train, vecs2), axis=1)

df_train.shape

In [ ]:
# df_train.to_pickle("data_vec.pkl")

In [ ]:
%reset

In [ ]:
df_train = pd.read_pickle("data_vec.pkl")

In [ ]:
df_train.shape

In [ ]:
df_train.columns

In [ ]:
df_train.drop(['id', 'qid1', 'qid2', 'question1', 'question2', 'q1_s', 'q2_s'], axis=1, inplace=True)

In [ ]:
Y = df_train['is_duplicate']

In [ ]:
df_train.drop(['is_duplicate'], inplace=True, axis=1)

In [ ]:
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

In [ ]:
df_train = normalize(df_train)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train, Y, test_size=0.20, random_state=42)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)

In [ ]:
X_train, y_train = ros.fit_resample(X_train, y_train)

In [ ]:
y_train.value_counts()

In [ ]:
clf = LogisticRegression(class_weight={0: 0.4, 1: 0.6}, max_iter=300)
# clf = RandomForestClassifier()
# clf = BernoulliNB()
# clf = GradientBoostingClassifier()
# clf = KNeighborsClassifier()

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_test)

In [ ]:
print("Accuracy : ", accuracy_score(y_test, preds))
print("Precision : ", precision_score(y_test, preds))
print("Recall : ", recall_score(y_test, preds))

In [ ]:
conf_matrix(y_test, preds)

### try using lstm, gru, fasttext embedding

In [27]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

In [8]:
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'

In [9]:
df_train = pd.read_csv('train.csv')

In [10]:
train_df, test_df = train_test_split(df_train)

In [11]:
vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [17]:
stops = set(stopwords.words('english'))
def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

In [18]:
questions_cols = ['question1', 'question2']

# Iterate over the questions only of both training and test datasets
for dataset in [train_df, test_df]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):
#             for word in row[question].split():

                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            # Replace questions as word to question as number representation
#             dataset.set_value(index, question, q2n)
            dataset.at[index, question] = q2n
            
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

# del word2vec

In [30]:
max_seq_length = max(train_df.question1.map(lambda x: len(x)).max(),
                     train_df.question2.map(lambda x: len(x)).max(),
                     test_df.question1.map(lambda x: len(x)).max(),
                     test_df.question2.map(lambda x: len(x)).max())

# Split to train validation
validation_size = 30000
training_size = len(train_df) - validation_size

X = train_df[questions_cols]
Y = train_df['is_duplicate']

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)

X_train, Y_train = ros.fit_resample(X_train, Y_train)
# Split to dicts
X_train = {'left': X_train.question1, 'right': X_train.question2}
X_validation = {'left': X_validation.question1, 'right': X_validation.question2}
X_test = {'left': test_df.question1, 'right': test_df.question2}

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

In [ ]:
# Model variables
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 32
n_epoch = 1

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])

# Start training
training_start_time = time()

malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

/home/prerit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 344642 samples, validate on 30000 samples
Epoch 1/1
344640/344642 [============================>.] - ETA: 0s - loss: 0.5365 - accuracy: 0.7479 - precision_5: 0.7509 - recall_4: 0.7420

In [ ]:
# Plot accuracy
plt.plot(malstm_trained.history['acc'])
plt.plot(malstm_trained.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()